In [1]:
import pandas as pd

In [18]:
data = pd.read_csv("iri15.csv")

In [19]:
cl = data.columns.to_list()
clu = [c for c in cl if 'unit.1' in c]
clh = [c for c in cl if 'holiday.1' in c]


In [12]:
data[clu]

,unit.1424,unit.1425,unit.1426,unit.1427,unit.1428,unit.1429,unit.1430,unit.1431,unit.1432,unit.1433,...,unit.1469,unit.1470,unit.1471,unit.1472,unit.1473,unit.1474,unit.1475,unit.1476,unit.1477,unit.1478
0,6,11,2,2,3,5,7,5,10,9,...,8,12,11,14,10,13,3,6,6,1
1,30,22,9,23,13,15,25,27,41,21,...,25,13,8,9,13,15,15,14,4,11
2,22,25,10,27,17,24,20,21,18,28,...,20,41,34,21,22,35,32,39,31,9
3,9,3,8,7,4,4,4,1,3,7,...,6,5,4,4,5,4,4,6,7,3
4,2,3,8,8,14,4,13,7,3,8,...,5,4,5,8,6,1,5,8,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6935,20,14,6,14,33,19,9,23,31,31,...,4,6,14,11,6,8,6,7,10,6
6936,28,61,18,20,27,29,21,14,18,26,...,8,3,6,2,3,3,3,5,5,4
6937,4,6,2,10,6,3,3,2,2,3,...,2,7,5,4,6,4,3,4,3,12
6938,54,112,47,42,38,29,25,48,33,29,...,36,51,28,42,68,40,59,52,81,70


In [13]:
data[clh]

,holiday.1424,holiday.1425,holiday.1426,holiday.1427,holiday.1428,holiday.1429,holiday.1430,holiday.1431,holiday.1432,holiday.1433,...,holiday.1469,holiday.1470,holiday.1471,holiday.1472,holiday.1473,holiday.1474,holiday.1475,holiday.1476,holiday.1477,holiday.1478
0,0,0,1,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
1,0,0,1,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
2,0,0,1,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
3,0,0,1,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
4,0,0,1,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6935,0,0,1,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
6936,0,0,1,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
6937,0,0,1,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
6938,0,0,1,1,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1


In [16]:
data[clu]

,unit.1417,unit.1418,unit.1419,unit.1420,unit.1421,unit.1422,unit.1423,unit.1424,unit.1425,unit.1426,...,unit.1462,unit.1463,unit.1464,unit.1465,unit.1466,unit.1467,unit.1468,unit.1469,unit.1470,unit.1471
0,5,7,11,9,7,5,6,6,11,2,...,13,9,7,11,3,10,8,8,12,11
1,22,9,16,15,6,15,33,30,22,9,...,31,13,4,19,17,24,13,25,13,8
2,28,14,30,15,15,15,14,22,25,10,...,18,19,21,36,17,40,32,20,41,34
3,3,5,4,3,4,2,3,9,3,8,...,3,5,6,8,11,6,8,6,5,4
4,13,4,3,6,8,6,4,2,3,8,...,11,2,6,6,8,2,1,5,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222,7,15,14,19,8,23,21,20,14,6,...,5,15,7,9,4,5,2,4,6,14
7223,11,12,53,16,22,7,28,28,61,18,...,4,8,10,3,3,5,2,8,3,6
7224,8,4,3,3,6,3,1,4,6,2,...,3,4,7,6,4,6,3,2,7,5
7225,35,38,43,37,66,41,42,54,112,47,...,38,42,38,33,46,42,36,36,51,28


In [17]:
data[clh]

,holiday.1417,holiday.1418,holiday.1419,holiday.1420,holiday.1421,holiday.1422,holiday.1423,holiday.1424,holiday.1425,holiday.1426,...,holiday.1462,holiday.1463,holiday.1464,holiday.1465,holiday.1466,holiday.1467,holiday.1468,holiday.1469,holiday.1470,holiday.1471
0,0,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
1,0,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,0,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
3,0,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7222,0,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
7223,0,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
7224,0,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
7225,0,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0


In [20]:
data[clu]

,unit.1326,unit.1327,unit.1328,unit.1329,unit.1330,unit.1331,unit.1332,unit.1333,unit.1334,unit.1335,...,unit.1371,unit.1372,unit.1373,unit.1374,unit.1375,unit.1376,unit.1377,unit.1378,unit.1379,unit.1380
0,23,24,23,31,32,26,29,23,30,32,...,20,19,19,11,19,21,24,18,24,17
1,12,13,17,20,12,6,16,10,8,15,...,14,23,19,9,11,27,19,24,27,30
2,1,1,5,2,2,5,1,1,5,6,...,9,10,10,7,10,6,10,10,13,4
3,1,2,13,13,6,3,15,13,15,10,...,7,12,15,11,13,9,3,2,3,17
4,46,42,41,34,44,32,42,45,28,27,...,23,26,32,32,16,40,56,49,42,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8495,24,14,22,22,28,22,26,12,21,12,...,21,28,20,24,26,25,19,15,16,8
8496,1637,2601,1593,1618,2483,1968,1682,1772,2040,2278,...,1712,1832,2174,1594,1363,1696,1699,1572,1704,1415
8497,259,232,232,248,238,246,225,227,291,197,...,209,461,235,216,218,220,267,232,251,219
8498,130,113,99,121,91,112,120,123,158,74,...,102,118,173,102,110,128,111,131,124,97


In [21]:
data[clh]

,holiday.1326,holiday.1327,holiday.1328,holiday.1329,holiday.1330,holiday.1331,holiday.1332,holiday.1333,holiday.1334,holiday.1335,...,holiday.1371,holiday.1372,holiday.1373,holiday.1374,holiday.1375,holiday.1376,holiday.1377,holiday.1378,holiday.1379,holiday.1380
0,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8495,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
8496,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
8497,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
8498,0,0,0,1,0,0,0,0,1,0,...,0,0,1,1,0,0,0,0,0,0
